In [42]:
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import sys
import pickle as pkl
import face_recognition
sys.path.append('..')

from process_latents import ks
import data
import style
import config
from config import *
import util

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### celeba-hq

In [153]:
# look at ims
ims = []
for IM_NUM in np.arange(1, 91):
    
    ims.append(mpimg.imread(oj(DIR_IMS, f'{IM_NUM:05}.jpg')))

images = np.array(ims)
images = images.reshape(9, 10, *images.shape[1:])

if type(images) == list:
    images = np.array(images)
N_IMS = images.shape[0]
R = images.shape[0]
C = images.shape[1]
i = 0
plt.figure(figsize=(C * 3, R * 3))
for r in range(R):
    for c in range(C):
        plt.subplot(R, C, i + 1)
        util.imshow(images[r, c])
        plt.title(str(df['gender'].values[i]))

        i += 1

plt.subplot(R, C, N_IMS // 2 + 1)
plt.tight_layout()

In [37]:
regs = [0, 0.01, 0.1, 1, 10000] # there are also some for 0.01, 10 -- these weren't as good so let's ignore these
IM_NUMS = np.arange(10)

In [ ]:
R, C = len(IM_NUMS), 1 + len(regs)
plt.figure(dpi=120, figsize=(C*2.5, R*2.5))
for r, IM_NUM in enumerate(IM_NUMS):
    ims = []
    latents = []
    im_orig = mpimg.imread(oj(DIR_IMS, f'{IM_NUM:05}.jpg'))
    for reg in regs:
        folder = f'generated_images_{reg}'
        im_fname = oj(DIR_PROCESSED, folder, f'{IM_NUM:05}.png')
        ims.append(mpimg.imread(im_fname))
        latents.append(np.load(oj(DIR_PROCESSED, folder, f'{IM_NUM:05}.npy')))
        # print(np.mean(np.abs(np.corrcoef(latents[-1]))))

    
    plt.subplot(R, C, C * r + 1)
    util.imshow(im_orig)
    if r == 0:
        plt.title('orig', fontsize=9)

    for i in range(len(regs)):
        plt.subplot(R, C, C * r + 2 + i)
        util.imshow(ims[i])
        if r == 0:
            plt.title(f'reg={regs[i]:.2e}', fontsize=9)
    latents = np.array(latents)
plt.tight_layout()

### stylegan2-generated

In [2]:
DIR_ORIG = '../data/annotation-dataset-stylegan2/images'
DIRS_STYLEGAN = '../data_processed/stylegan2'

In [3]:
regs = [0, 0.1, 1, 10000]
IM_NUMS = [0, 1, 10]

In [ ]:
plt.figure(dpi=300)
R, C = len(IM_NUMS), 1 + len(regs)
for r, IM_NUM in enumerate(IM_NUMS):
    ims = []
    latents = []
    im_orig = mpimg.imread(oj(DIR_ORIG, f'{IM_NUM}.jpg'))
    for reg in regs:
        folder = f'generated_images_{reg}'
        im_fname = oj(DIRS_STYLEGAN, folder, f'{IM_NUM}.png')
        ims.append(mpimg.imread(im_fname))
        latents.append(np.load(oj(DIRS_STYLEGAN, folder, f'{IM_NUM}.npy')))
        print(np.mean(np.abs(np.corrcoef(latents[-1]))))

    
    plt.subplot(R, C, C * r + 1)
    util.imshow(im_orig)

    for i in range(len(regs)):
        plt.subplot(R, C, C * r + 2 + i)
        util.imshow(ims[i])

### celeba (not hq)

In [5]:
DIR_ORIG_CELEBA = '../data/CelebA/Img/img_align_celeba'
DIR_SQUARE_CELEBA = '../data/CelebA/Img/img_square_celeba'

**need to first preprocess celeba to be square before running the pipeline**

In [ ]:
for fname in tqdm(sorted(os.listdir(DIR_ORIG_CELEBA))):
    if '.jpg' in fname:
        im = mpimg.imread(oj(DIR_ORIG_CELEBA, fname))
        if not im.shape[0] == im.shape[1]:
            im = im[20:-20]
        plt.imsave(oj(DIR_SQUARE_CELEBA, fname), im)

## misc images

In [ ]:
dir_base = '../data/pilot-our-images-large/'
dir_out_base = '../data_processed/pilot-our-images-large'
for folder in os.listdir(dir_out_base):
    dir_in = oj(dir_base, folder)
    dir_out = oj(dir_out_base, folder)
    print(folder)
    ims = []
    for fname in sorted(os.listdir(dir_out)):
        if '.jpg' in fname or '.png' in fname:
            img_path = oj(dir_out, fname)
            ims.append(mpimg.imread(img_path))
    util.plot_grid(ims)
    plt.show()